<a href="https://colab.research.google.com/github/shiwkumar/UAV_CR/blob/main/Game_Theory_Stackelberg_Game_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd1
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
from tqdm import tqdm
import h5py

class StackelbergGame:
    def __init__(self, num_secondary_users=4):
        self.num_su = num_secondary_users
        self.sensing_time = 0.001  # 1ms sensing time
        self.sampling_rate = 40e6  # 40MHz sampling rate
        self.samples_per_sensing = int(self.sensing_time * self.sampling_rate)
        self.noise_variance = 1.0
        self.price_coefficient = 0.1
        self.energy_coefficient = 0.01

    def leader_utility(self, price, detection_prob):
        """Primary user (leader) utility function"""
        return price * detection_prob - self.price_coefficient * price**2

    def follower_utility(self, detection_prob, sensing_time, price):
        """Secondary user (follower) utility function"""
        energy_cost = self.energy_coefficient * sensing_time
        return detection_prob - price * sensing_time - energy_cost

    def calculate_detection_probability(self, snr, sensing_time):
        """Calculate probability of detection using SNR and sensing time"""
        samples = int(sensing_time * self.sampling_rate)
        Q = lambda x: 0.5 * (1 - norm.cdf(x/np.sqrt(2)))
        pf = 0.1  # Fixed false alarm probability

        lambda_thresh = self.noise_variance * (1 + np.sqrt(2/samples) * norm.ppf(1-pf))
        gamma = 10**(snr/10)
        pd = Q((lambda_thresh/self.noise_variance - gamma - 1) * np.sqrt(samples/2/(2*gamma + 1)))
        return pd

    def optimize_price(self, snr):
        """Optimize price for primary user"""
        prices = np.linspace(0.1, 2, 100)
        max_utility = -np.inf
        optimal_price = prices[0]

        for price in prices:
            sensing_time = self.optimize_sensing_time(price, snr)
            pd = self.calculate_detection_probability(snr, sensing_time)
            utility = self.leader_utility(price, pd)

            if utility > max_utility:
                max_utility = utility
                optimal_price = price

        return optimal_price

    def optimize_sensing_time(self, price, snr):
        """Optimize sensing time for secondary users"""
        sensing_times = np.linspace(0.1e-3, 2e-3, 100)
        max_utility = -np.inf
        optimal_time = sensing_times[0]

        for time in sensing_times:
            pd = self.calculate_detection_probability(snr, time)
            utility = self.follower_utility(pd, time, price)

            if utility > max_utility:
                max_utility = utility
                optimal_time = time

        return optimal_time

def run_simulation(dataset_path, simulation_time=60):
    """Run spectrum sensing simulation for 60 seconds"""
    # Load dataset
    df = pd1.read_hdf(dataset_path)

    # Initialize parameters
    game = StackelbergGame(num_secondary_users=4)
    sensing_interval = 0.001  # 1ms
    num_iterations = int(simulation_time / sensing_interval)
    snr_range = np.arange(-20, 6, 1)

    # Initialize results storage
    detection_probabilities = {snr: [] for snr in snr_range}
    price_history = []
    sensing_time_history = []

    # Run simulation
    print("\nRunning Stackelberg Game simulation...")
    for snr in tqdm(snr_range):
        # Filter dataset for current SNR
        snr_data = df[np.abs(df['snr'] - snr) < 0.5]

        if len(snr_data) > 0:
            # Optimize price and sensing time
            optimal_price = game.optimize_price(snr)
            optimal_sensing_time = game.optimize_sensing_time(optimal_price, snr)

            # Calculate detection probability
            pd = game.calculate_detection_probability(snr, optimal_sensing_time)

            detection_probabilities[snr].append(pd)
            price_history.append(optimal_price)
            sensing_time_history.append(optimal_sensing_time)

    # Process results
    avg_detection_prob = {snr: np.mean(probs) if probs else 0
                         for snr, probs in detection_probabilities.items()}

    # Create visualization
    plt.figure(figsize=(12, 8))

    # Plot detection probability vs SNR
    snrs = list(avg_detection_prob.keys())
    pds = list(avg_detection_prob.values())

    plt.plot(snrs, pds, 'b-', linewidth=2, label='Stackelberg Game')

    # Add theoretical curve for comparison
    theoretical_pd = [game.calculate_detection_probability(snr, np.mean(sensing_time_history))
                     for snr in snrs]
    plt.plot(snrs, theoretical_pd, 'r--', linewidth=2, label='Theoretical')

    plt.grid(True)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Probability of Detection')
    plt.title('Spectrum Sensing Performance using Stackelberg Game\n'
              f'(Number of SUs: {game.num_su}, Simulation Time: {simulation_time}s)')
    plt.legend()
    plt.ylim([0, 1])

    # Save plot
    plt.savefig('stackelberg_simulation_results.png')
    plt.close()

    # Print results
    print("\nSimulation Results:")
    print("------------------")
    print(f"Average Detection Probability: {np.mean(list(avg_detection_prob.values())):.4f}")
    print(f"Average Price: {np.mean(price_history):.4f}")
    print(f"Average Sensing Time: {np.mean(sensing_time_history)*1000:.2f} ms")

    # Save detailed results
    results = {
        'snr': snrs,
        'detection_probability': pds,
        'theoretical_pd': theoretical_pd,
        'price_history': price_history,
        'sensing_time_history': sensing_time_history
    }

    return results

if __name__ == "__main__":
    # Run simulation
    results = run_simulation('cr_uav_dataset_10k.h5')

    # Save results to file
    np.savez('stackelberg_simulation_results.npz', **results)

    # Create additional analysis plots
    plt.figure(figsize=(15, 5))

    # Price history
    plt.subplot(121)
    plt.plot(results['snr'], results['price_history'], 'g-')
    plt.grid(True)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Optimal Price')
    plt.title('Price vs SNR')

    # Sensing time history
    plt.subplot(122)
    plt.plot(results['snr'], np.array(results['sensing_time_history'])*1000, 'm-')
    plt.grid(True)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Optimal Sensing Time (ms)')
    plt.title('Sensing Time vs SNR')

    plt.tight_layout()
    plt.savefig('stackelberg_additional_analysis.png')
    plt.close()